# RQ4 - Prepare Historical Coocurrence - Last slice

Uses:

- resources/historical_join.xlsx (HISTORICAL_FILE_JOIN from prepare/1.historical.ipynb)

Generates:

- resources/historical_rulesv10.xlsx


In [1]:
import sys  
sys.path.insert(1, '..')

In [2]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)
df

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,0,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects have DB clues

In [4]:
df2 = df.groupby(["owner", "name"]).sum()
df2.drop(["part_commit", "isLast"], axis=1, inplace=True)
df2.drop(df2[(df2 == 0).all(axis=1)].index, inplace=True)
df2

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,0,0,0,0,0,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Athou,commafeed,0,0,0,0,0,0,10,0,0,0,...,0,0,0,0,0,18,0,0,0,0
Atmosphere,atmosphere,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,0,0,0,0,0,0,203,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xuxueli,xxl-job,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New dataframe filtering only lines that are in the last slice of the projects

In [5]:
# Função para selecionar o último commit
def select_last_commit(group):
    return group.iloc[-1]  # Seleciona o último commit

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df[~df["isLast"]].sort_values(by=['owner', 'name', 'part_commit'])

# Aplica a função para pegar o último commit em cada grupo de projeto
df10 = df_sorted.groupby(['owner', 'name']).apply(select_last_commit)
df10.drop(["owner", "name", "domain", "sha1", "part_commit", "date_commit", "isLast"], axis=1, inplace=True)
df10

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
AlmasB,FXGL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,AntennaPod,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xuxueli,xxl-job,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yacy,yacy_search_server,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaproxy,zaproxy,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the last slice 

In [6]:
with_dbs = df10.loc[df2.index].copy().astype('bool')
with_dbs

,,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Alluxio,alluxio,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
AnySoftKeyboard,AnySoftKeyboard,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Athou,commafeed,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Atmosphere,atmosphere,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wildfly,wildfly,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
xuxueli,xxl-job,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
yacy,yacy_search_server,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### ASSOCIATION RULES

In [7]:
frq_items = apriori(with_dbs, min_support = 0.021, use_colnames = True)

In [8]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

In [9]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending=False)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len,consequente_len
362956,"(SAP Adaptive Server, Oracle, SQLite)","(H2, MySQL, Informix, MS SQL Server, PostgreSQ...",0.021459,0.025751,0.021459,1.000000,38.833333,0.020907,inf,0.995614,3,7
334240,"(H2, SAP Adaptive Server, HyperSQL, SQLite)","(Informix, MS SQL Server, IBM DB2, Oracle, Fir...",0.021459,0.025751,0.021459,1.000000,38.833333,0.020907,inf,0.995614,4,5
364507,"(H2, Informix, MS SQL Server, Firebird, HyperSQL)","(MySQL, SQLite, SAP Adaptive Server, IBM DB2, ...",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,5,5
364505,"(H2, Informix, MS SQL Server, PostgreSQL, Fire...","(MySQL, SQLite, SAP Adaptive Server, IBM DB2, ...",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,5,5
364503,"(H2, Informix, MS SQL Server, IBM DB2, Firebird)","(MySQL, SQLite, SAP Adaptive Server, PostgreSQ...",0.025751,0.021459,0.021459,0.833333,38.833333,0.020907,5.871245,1.000000,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
18368,(Redis),"(H2, MySQL, SQLite)",0.347639,0.072961,0.025751,0.074074,1.015251,0.000387,1.001202,0.023026,1,3
65419,(Redis),"(H2, MySQL, SQLite, PostgreSQL)",0.347639,0.072961,0.025751,0.074074,1.015251,0.000387,1.001202,0.023026,1,4
18357,"(H2, MySQL, SQLite)",(Redis),0.072961,0.347639,0.025751,0.352941,1.015251,0.000387,1.008194,0.016204,3,1
19506,"(H2, SQLite, PostgreSQL)",(Redis),0.072961,0.347639,0.025751,0.352941,1.015251,0.000387,1.008194,0.016204,3,1


Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [10]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len,consequente_len
369,(Ingres),(Firebird),0.021459,0.047210,0.021459,1.000000,21.181818,0.020446,inf,0.973684,1,1
368,(Firebird),(Ingres),0.047210,0.021459,0.021459,0.454545,21.181818,0.020446,1.793991,1.000000,1,1
44,(SAP Adaptive Server),(Informix),0.034335,0.042918,0.025751,0.750000,17.475000,0.024277,3.828326,0.976296,1,1
45,(Informix),(SAP Adaptive Server),0.042918,0.034335,0.025751,0.600000,17.475000,0.024277,2.414163,0.985052,1,1
32,(Informix),(MS Access),0.042918,0.030043,0.021459,0.500000,16.642857,0.020170,1.939914,0.982063,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
152,(MySQL),(Hazelcast),0.480687,0.072961,0.042918,0.089286,1.223739,0.007847,1.017925,0.352066,1,1
211,(HBase),(H2),0.064378,0.386266,0.030043,0.466667,1.208148,0.005176,1.150751,0.184142,1,1
210,(H2),(HBase),0.386266,0.064378,0.030043,0.077778,1.208148,0.005176,1.014530,0.280719,1,1
193,(DynamoDB),(H2),0.124464,0.386266,0.055794,0.448276,1.160536,0.007718,1.112393,0.157994,1,1


Saving Results

In [11]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv10.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)